# Promise

<img src="img/hero.jpg" style="{ display: inline-block; width: 70%; }" >

> 説到「Promise」，你可能會覺得很困惑。承諾？為什麼 JS 會有 _承諾_ 這個東西？

筆者認為，要體會和理解 Promise ，第一個要搞懂的事情是「非同步」。

---

首先，我們要先對於「非同步執行」的程式碼有相當程度的體會。

讓我們來看看一些程式範例。

## 同步執行

同步執行的概念就是你最習慣的 JS 執行方式：

> 先做第一行，等完成之後再做第二行，以此類推

讓我們來看看一個簡單的範例，

在執行之前，猜看看，結果會是什麼呢？

In [ ]:
var msg = 'Hello';

console.log(msg);

msg = 'Byebye';

console.warn(msg);

## 非同步執行

非同步執行的概念就是打破你習慣的『同步執行』

> 先做第一行，但是 __第一行還沒有完成__ 的時候，第二行就開始做了

這個範例和上一個有一些地方不同，是哪裡呢？

結果和你預期的一樣嗎？

In [ ]:
function greetAsync() {
  setTimeout(function() {
    console.log(msg);
  }, 1000);
}

var msg = 'Hello';

greetAsync();

msg = 'Byebye';

console.warn(msg);

上面的範例中，可以看到我們建立了一個新函式 `greetAsync()`，

  - 將 `console.log(msg)` 被包在函式裡，放到了 [`setTimeout`][setTimeout] 裡面，設定一秒後執行。

同時，我們也可以從結果看出，JS 引擎並不會等第七行完成，

  - 而是先繼續執行後面的第九行和第十一行，最後才執行第七行。

  - 第七行執行時，變數 `msg` 的值被改成了 `'ByeBye'`，被印出來。

所以，第七行的 `greetAsync` 就是所謂的「非同步函式」。它的執行是非同步的（或稱作異步執行）。


> 所以我們也可以說，`setTimeout()` 是個 JS 內建的「非同步執行」的例子。

[setTimeout]: https://developer.mozilla.org/en-US/docs/Web/API/WindowOrWorkerGlobalScope/setTimeout

### 非同步執行的好處與壞處

上面的例子有個顯而易見的壞處，就是『維護不易』。

我們很難一眼從程式碼本身就看出它執行的結果。

或者是說，我們也很難預期非同步的程式碼會在什麼時候執行。

而好處是，現代網頁中有許多操作是『費時且無法預期結果』的。

像是__藉由網路送出請求__ (我們無法確定它何時會完成)。

如果，這些操作是同步的，會造成網頁體驗上有很大的缺陷。

（請想像，網路連線緩慢時，你按下按鈕要等幾秒畫面才告訴你結果）

因此，在現在網頁開發的趨勢之下，前端工程師的命運就是，

必須要 __學會非同步執行的程式碼__。

而 Promise 帶來了一些針對非同步執行的標準，能減輕我們一些負擔。

## 一個 Promise 代表著一個非同步的執行

體驗完了同步與非同步程式碼的行為，我們終於可以來聊聊我們的主角了 -- `Promise`。

> 一個 Promise 物件代表一個「非同步函式」的執行。
> 未來可以期待這個函式會回傳結果，這是 JS 引擎給你的承諾。

讓我們來看看一個簡單的範例：

In [ ]:
function greetPromise() {
  return new Promise(function() {
    setTimeout(function() {
      console.log(msg);
    }, 1000);
  });
}

var msg = 'Hello';

var myFirstPromise = greetPromise();

msg = 'Byebye';

console.warn(msg);

// what would it be?
// console.log(myFirstPromise)

可以看到我們在第九行利用 `greetPromise()` 建立了一個 Promise 物件 `myFirstPromise`，

- 它代表著一個承諾，表示未來才會有結果的執行
- 所以，當執行到第十一行時，第九行的結果還沒有產生

試看看：移除最後的註解，把 `myFirstPromise` 印出來看看會是什麼？

### 那，這裡用 promise 物件到底有什麼目的呢？

> 因為我們常常需要「等非同步操作完成」

在之前使用 `greetAsync()` 的範例中，我們無法等它有執行結果，再執行第九行將 `msg` 改成 `'ByeBye'`

除非你將第九行以後的程式碼放進 `setTimeout`。然而，

> 真實世界中還有其他非同步操作

這裡的 `setTimeout()` 只是用來模擬一種固定會在一秒後有結果的操作，

在真實世界中，送出 HTTP 請求就是個「不確定什麼時候會完成」的操作。

--- 

所以，我們需要一個機制來讓 _非同步的執行結果能決定程式執行的流程_

## Promise 木偶，我要怎麼操控你？

加上真實世界的非同步執行太複雜，我們需要公用的標準來處理。

  - Promise 其實就是針對「[非同步函式管理的標準][a-plus]」的實現。其中一個概念大概是這麼說：
  

> 非同步執行函式的狀態，可能是「等待中」，不然就是帶有回傳值的「已實現」，
> 不然就是帶有回傳理由的「已失敗」。

<!-- 狀態示意圖 -->
  

[a-plus]: https://promisesaplus.com/

筆者想讚嘆：使用三種狀態來表示非同步執行是非常精巧的設計。以「向伺服器送出 HTTP 請求」為例：

- 「等待中」可能代表他正在和伺服器建立連線，或是等待回應 (pending)
- 「已實現」表示該請求收到回應，回傳值就是 HTTP 回應 (fulfilled)
- 「已失敗」表示出現例外情況的錯誤，像是連線逾時，回傳理由可以是錯誤物件 (rejected)

另外，promise 的變化永遠是單方向的：

1. 一開始是 pending
2. 接下來只可能會變成 fulfilled，或是變成 rejected
3. 之後就再也不會改變了 (settled, 固定的)

這對開發者真是好消息。每個 promise 只要處理兩種可能(2)，而且需要處理一次(3)。

所以，常見的 promise 練習範例就是處理 `Promise` 的兩種 settled 狀態 - 「已實現」或是「已失敗」。

讓我們來看看怎麼做

In [ ]:
var fetch = require('node-fetch'); // require $ npm i -S node-fetch
var fetchJson = function(url) {
  return fetch(url).then(function(resp) {
    return resp.json();
  });
};

var myFetch = fetchJson('https://api.github.com/users/github');
myFetch.then(function(json) {
  console.warn(json);
});

console.log('fetching data ...');

重點在第四行，我們用 `fetchJson()` 建立一個 promise 物件 `myFetch`
  - 它的非同步工作是「向 GitHub api 送出請求，請求使用者資料」

然後我們使用了 `myFetch.then()` 來處理 `myFetch` 的「已實現」狀態

- 用法是 `myFetch.then(myHandler)`, `myHandler` 是執行處理的函式，第一個參數就會是 HTTP 回應的 body
- 可以注意到執行順序，會是先第七行，然後等到請求「已實現」後，才會到第五行，印出使用者資料

_試看看：：如果輸入錯誤的網址，會有什麼結果呢？_

讓我們來看看一個處理「已失敗」的例子

In [ ]:
var fetch = require('node-fetch');
var fetchJson = function(url) {
  return fetch(url).then(function(resp) {
    return resp.json();
  });
};

var myFailFetch = fetchJson('ftp://api.github.com/users/github');
myFailFetch.catch(console.error);

console.log('fetching data ...');

// what would happen if doing myFailFetch.then(console.error) ?

可以看到，由於使用了錯誤的通訊協定 `ftp` ，所以 `myFailFetch` 這個 promise 的執行失敗

我們使用 `myFailFetch.catch()` 來處理「已失敗」狀態

- 非常相似，用法是 `myFailFetch.then(myHandler)`, `myHandler` 是個函式，第一個參數就會是錯誤物件
- 執行順序一樣，會是先第七行，然後等到請求「已失敗」後，才會到第五行，印出錯誤

_試看看：：`myFailFetch.then(console.error)` 會有什麼結果？_

---

我們可以從 `myFailFetch.then(...)` 的實驗中發現，
  - 一個狀態是「已失敗」的 promise 是不會執行 `.then()` 裡面的 handler 的。

這個特性很重要，我們之後會來回顧這個現象。


## Promise 與鐵鍊

<img src="img/chain.jpg" style="{ display: inline-block; width: 70%; }" >

讓我們來看看一個有趣的問題：

> `myFailFetch.then()` 和 `myFailFetch.catch()` 回傳的值會是什麼？

In [ ]:
var fetch = require('node-fetch');
var fetchJson = function(url) {
  return fetch(url).then(function(resp) {
    return resp.json();
  });
};
var doNothing = function() {};

var myFetch = fetchJson('https://api.github.com/users/github');
var x = myFetch.then(doNothing);
console.error(x);
console.log(x === myFetch);

var myFailFetch = fetchJson('ftp://api.github.com/users/github');
var y = myFailFetch.catch(doNothing);
console.error(y);
console.log(y === myFailFetch);

// what's the status of myFailFetch? what's the resolve value of it?

印出了 `.catch()` 回傳的 promise 之後，你有注意到什麼有趣的地方嗎？

- 使用了 `.then` 或 `.catch` 後，回傳了 _另一個不同的_ promise 物件

這就代表著 `.then()` `.catch()` 是可以「接連著呼叫」於一個起始 promise 物件之後。

而且各個 handler 的執行是有順序的。像這樣：

```js
var somePromise = myFetch.then(console.log);
somePromise.then(doSomething);
```

會依序執行 `console.log` 再執行 `doSomething`。
可以再把變數抽掉，縮寫成：

```js
myFetch.then(console.log).then(doSomething);
```


> 這樣的模式叫做 _Promise chaining_。你可以讓數個 promise handler 有順序的執行。

---

而經過這樣連環呼叫後的 promise 會有什麼特色呢？我要怎麼用它？

根據 [Promise 文件][doc]：

> `.then(myHandler)` 和 `.catch(myHandler)` 回傳的 promise 的狀態 X 和其回傳值 Y，取決於 __`myHandler` 的執行和回傳值__

1. `myHandler` 執行的回傳值如果 _不是 promise_ ，則 X 就是「已實現」，回傳值 Y 就是 `myHandler` 回傳值
2. `myHandler` 執行的回傳值如果 _是個 promise_ ，則 X 和 Y 就由該 promise 決定
3. `myHandler` 執行時 _拋錯_ ，則 X 就是「已失敗」，回傳值 Y 就是 `myHandler` 拋的錯誤

[doc]: https://developer.mozilla.org/zh-TW/docs/Web/JavaScript/Reference/Global_Objects/Promise/then

讓我們來看看一個例子

In [ ]:
var fetch = require('node-fetch');
var fetchJson = function(url) {
  return fetch(url).then(function(resp) {
    return resp.json();
  });
};

var myFailFetch = fetchJson(
  'https://api.github.com/repos/jupyter/notebook/contributors'
);
myFailFetch
  .then(function(contributors) {
    return contributors[0];
  })
  .then(function(user) {
    return fetchJson('https://api.github.com/users/' + user.login);
  })
  .then(function(user) {
    return console.error(user.login, user.bio);
  });

console.log('fetching data ...');

// how can we test the `.catch()`?

可以看到我們從 `myFailFetch` 出發，使用了 3 個 handler 將他們依序連接，像是鐵鍊那樣。

如果將它們編號為 1 到 3 號，概念圖畫起來會是這樣：

`.then(handler_1)` --> `.then(handler_2)` --> `.then(handler_3)`

但是，這樣的情況其實是 _要符合某些條件的_。條件是

> 每個 handler 執行後，__套用了先前提到的三個判斷條件條件__，回傳的 _新 promise_ 的狀態是「已解決」

想看看，如果 
  - `handler_1` 執行時拋錯(1)
  - `handler_2` 回傳的 promise `fetchJson(...)` 遇到網路問題，狀態是「已失敗」(2)

最後都會產生「已失敗」的 promise，讓後面的 `.then()` 不會執行。被「斷開鎖鏈」了。

考慮上述情況後，更完整的概念圖：

![unhanlded-chain](img/unhandled-chain.png)

為了要處理例外情況造成「已失敗」promise ，這裡會介紹一個常見的 promise chaining 實踐，就是

> 在鎖鏈裡後方放一個 `.catch`

In [ ]:
var fetch = require('node-fetch');
var fetchJson = function(url) {
  return fetch(url).then(function(resp) {
    return resp.json();
  });
};

var myFailFetch = fetchJson(
  'https://api.github.com/repos/jupyter/notebook/contributors'
);
myFailFetch
  .then(function(contributors) {
    return contributors[0];
  })
  .then(function(user) {
    return fetchJson('https://api.github.com/users/' + user.login);
  })
  .catch(function(err) {
    return { login: 'NOT-FOUND', bio: 'N/A' };
  })
  .then(function(user) {
    return console.error(user.login, user.bio);
  });

console.log('fetching data ...');

現在我們多加了一個 `.catch(errHandler)` 能處理 `handler_1` 或 `handler_2` 產生「已失敗」promise 的情況。

同時，`errHandler` 只是回傳一個物件，所以可以很有信心的說，它會產生一個「已解決」的 promise 給 `handler_3`。

這樣更能確保 `handler_3` 總是能夠執行。

最後圖畫起來會是這樣：

![handled-chain](img/handled-chain.png)

## 最後，下山前至少記得這三件事

如果你是第一次接觸 Promise 的開發者，請接受筆者衷心的敬佩，

你沈住氣，閱讀並嘗試理解許多新觀念，執行每個程式碼範例，挑戰實驗題，最後到了這篇文章的結尾。


如果說 Promise 技術知識是座山，你雖然沒有登上最高峰，

但你成功征服了一條步道，站在山腰上不錯的位置，欣賞風景。

---

希望未來你會想了解更多關於「非同步」JavaScript。

希望下山時，你還能記得這些事：

1. 一個 promise 代表著一個「非同步」的執行
2. promise 的兩個狀態「已實現」「已失敗」分別用 `.then()` 和 `.catch()` 來處理
3. promise handler 可以像鎖鏈一樣接著呼叫，handler 會造成 promise 有不同的狀態和回傳值

#### Ref

- [從Promise開始的JavaScript異步生活
](https://eyesofkids.gitbooks.io/javascript-start-es6-promise/content/contents/intro.html)

---

author: TC Liu <liuderchi@github>
date: June 18th 2018

---